# ETL Pipeline 

This notebook contains the Extract, Transform, Load (ETL) pipeline for the finance dataset.

## Pipeline Steps:
1. **Extract**: Load data from CSV file
2. **Transform**: Clean data and add enrichment features
3. **Load**: Save processed data to output file



## 

## Import Libraries

In [44]:
import pandas as pd
import numpy as np
import os

## Data Extraction

In [45]:
def extract_data(file_path):
    """Extract data from CSV file."""
    try:
        df = pd.read_csv(file_path)
        print(f"Data extracted successfully. Shape: {df.shape}")
        print("Columns:", df.columns.tolist())
        return df
    except Exception as e:
        print(f"Error extracting data: {e}")
        return None

## Data Cleaning

In [46]:
def clean_data(df):
    """Clean the dataset: handle missing values, standardize formats, remove outliers."""
    # Handle missing values
    numeric_cols = ['Stock_Price', 'Revenue_Millions', 'Net_Income_Millions', 'Market_Cap_Billions', 'EPS']
    for col in numeric_cols:
        df[col] = df[col].fillna(df[col].median())
    df['Company'] = df['Company'].fillna('Unknown')

    # Standardize date format
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

    # Remove outliers for Revenue_Millions using IQR
    Q1 = df['Revenue_Millions'].quantile(0.25)
    Q3 = df['Revenue_Millions'].quantile(0.75)
    IQR = Q3 - Q1
    df = df[(df['Revenue_Millions'] >= Q1 - 1.5 * IQR) & (df['Revenue_Millions'] <= Q3 + 1.5 * IQR)]

    print(f"Data cleaned. Shape after cleaning: {df.shape}")
    return df

## Data Enrichment

In [53]:
def enrich_data(df):
    """Add calculated fields: Profit_Margin, Revenue_Growth."""
    # Calculate profit margin
    df['Profit_Margin'] = (df['Net_Income_Millions'] / df['Revenue_Millions']) * 100

    # Calculate revenue growth (month-over-month percentage change per company)
    df = df.sort_values(['Company', 'Date'])
    df['Revenue_Growth'] = df.groupby('Company')['Revenue_Millions'].pct_change() * 100
    df['Revenue_Growth'] = df['Revenue_Growth'].fillna(0)

    print("Data enriched with Profit_Margin and Revenue_Growth.")
    return df

## Load Data

In [48]:
def load_data(df, output_path):
    """Save cleaned and enriched data to CSV."""
    # Create output directory if it doesn't exist
    output_dir = os.path.dirname(output_path)
    if output_dir:  # Only create directory if path contains a directory
        os.makedirs(output_dir, exist_ok=True)
    df.to_csv(output_path, index=False)
    print(f"Data saved to {output_path}")

In [49]:
# Main ETL pipeline
def run_etl(input_path, output_path):
    """Run the Week 2 ETL pipeline for cleaning and enrichment."""
    df = extract_data(input_path)
    if df is not None:
        df = clean_data(df)
        df = enrich_data(df)
        load_data(df, output_path)
    else:
        print("ETL pipeline failed due to extraction error.")

## Run ETL Pipeline and View Results

In [50]:
# Step 1: Load and examine the original data
print("=== ORIGINAL DATA ===")
original_data = extract_data('finance_dataset.csv')
if original_data is not None:
    print(f"\nOriginal data shape: {original_data.shape}")
    print("\nFirst 5 rows of original data:")
    print(original_data.head())
    
    print("\nData types:")
    print(original_data.dtypes)
    
    print("\nMissing values in original data:")
    print(original_data.isnull().sum())
    
    print("\nBasic statistics of original data:")
    print(original_data.describe())

=== ORIGINAL DATA ===
Data extracted successfully. Shape: (200, 7)
Columns: ['Date', 'Company', 'Stock_Price', 'Revenue_Millions', 'Net_Income_Millions', 'Market_Cap_Billions', 'EPS']

Original data shape: (200, 7)

First 5 rows of original data:
         Date    Company  Stock_Price  Revenue_Millions  Net_Income_Millions  \
0  2022-01-01   TechCorp       145.20           1200.50               170.25   
1  2022-01-01  HealthInc        80.10            850.75               110.50   
2  2022-01-01   RetailCo        42.50           1950.25                85.30   
3  2022-01-01    FinServ       110.75           1000.40               140.20   
4  2022-01-01   EnergyCo        65.30            700.60                90.15   

   Market_Cap_Billions  EPS  
0                72.60  2.3  
1                40.05  1.7  
2                21.25  1.1  
3                55.38  2.0  
4                32.65  1.5  

Data types:
Date                    object
Company                 object
Stock_Price      

In [51]:
# Step 2: Clean the data and examine the results
if original_data is not None:
    print("\n" + "="*50)
    print("=== CLEANING DATA ===")
    cleaned_data = clean_data(original_data.copy())
    
    print(f"\nCleaned data shape: {cleaned_data.shape}")
    print(f"Rows removed during cleaning: {original_data.shape[0] - cleaned_data.shape[0]}")
    
    print("\nFirst 5 rows of cleaned data:")
    print(cleaned_data.head())
    
    print("\nMissing values after cleaning:")
    print(cleaned_data.isnull().sum())
    
    print("\nData types after cleaning:")
    print(cleaned_data.dtypes)


=== CLEANING DATA ===
Data cleaned. Shape after cleaning: (199, 7)

Cleaned data shape: (199, 7)
Rows removed during cleaning: 1

First 5 rows of cleaned data:
        Date    Company  Stock_Price  Revenue_Millions  Net_Income_Millions  \
0 2022-01-01   TechCorp       145.20           1200.50               170.25   
1 2022-01-01  HealthInc        80.10            850.75               110.50   
2 2022-01-01   RetailCo        42.50           1950.25                85.30   
3 2022-01-01    FinServ       110.75           1000.40               140.20   
4 2022-01-01   EnergyCo        65.30            700.60                90.15   

   Market_Cap_Billions  EPS  
0                72.60  2.3  
1                40.05  1.7  
2                21.25  1.1  
3                55.38  2.0  
4                32.65  1.5  

Missing values after cleaning:
Date                   0
Company                0
Stock_Price            0
Revenue_Millions       0
Net_Income_Millions    0
Market_Cap_Billions    0
EP

In [ ]:
# Step 3: Enrich the data and examine the final results
if 'cleaned_data' in locals():
    print("\n" + "="*50)
    print("=== ENRICHING DATA ===")
    enriched_data = enrich_data(cleaned_data.copy())
    
    print(f"\nFinal enriched data shape: {enriched_data.shape}")
    print("\nFirst 5 rows of enriched data:")
    print(enriched_data.head())
    
    print("\nNew columns added:")
    new_columns = set(enriched_data.columns) - set(original_data.columns)
    print(list(new_columns))
    
    print("\nFinal data statistics:")
    print(enriched_data.describe())
    
    # Save the final data to processed_data folder as originally intended
    output_path = 'processed_data/cleaned_finance_dataset.csv'
    
    try:
        load_data(enriched_data, output_path)
    except PermissionError:
        print(f"Permission error: The file {output_path} might be open in another application.")
        print("Please close any applications that might have this file open (like Excel) and try again.")
        print("Or run this cell again after closing the file.")
    except Exception as e:
        print(f"Error saving file: {e}")
    
    print(f"\n=== SUMMARY ===")
    print(f"Original data: {original_data.shape[0]} rows, {original_data.shape[1]} columns")
    print(f"Final data: {enriched_data.shape[0]} rows, {enriched_data.shape[1]} columns")
    print(f"Rows removed: {original_data.shape[0] - enriched_data.shape[0]}")
    print(f"Columns added: {enriched_data.shape[1] - original_data.shape[1]}")


=== ENRICHING DATA ===
Data enriched with Profit_Margin and Revenue_Growth.

Final enriched data shape: (199, 9)

First 5 rows of enriched data:
         Date   Company  Stock_Price  Revenue_Millions  Net_Income_Millions  \
4  2022-01-01  EnergyCo         65.3            700.60                90.15   
9  2022-02-01  EnergyCo         66.1            710.25                92.30   
14 2022-03-01  EnergyCo         65.8            705.50                91.10   
19 2022-04-01  EnergyCo         66.5            715.75                93.25   
24 2022-05-01  EnergyCo         67.0            720.90                94.50   

    Market_Cap_Billions   EPS  Profit_Margin  Revenue_Growth  
4                 32.65  1.50      12.867542        0.000000  
9                 33.05  1.52      12.995424        1.377391  
14                32.90  1.51      12.912828       -0.668779  
19                33.25  1.53      13.028292        1.452870  
24                33.50  1.55      13.108614        0.719525  

